In [3]:
import numpy as np
import time
import matplotlib.pyplot as plt
from tqdm import tqdm

Utilizaremos los siguientes generadores de números uniformes:
- `Lineal congruential generator (LCG)`
- `XORShift (64 bits)`
- `Mersenne Twister (MT19937)`

In [4]:
class LCG:
    def __init__(self, seed):
        self.state = seed
        self.a = 16807
        self.m = 2**31 - 1
        self.c = 0
    
    def random(self):
        self.state = (self.state * self.a + self.c) % self.m
        return self.state / self.m # Normalize to [0, 1)

In [5]:
class Xorshift64:
    def __init__(self, seed):
        if seed == 0:
            raise ValueError("Seed must be non-zero")
        self.state = seed % (2**64)   
         
    def random(self):
        x = self.state
        # Apply the mask after each operation to keep it within 64-bit range
        x = (x ^ (x << 13)) & 0xFFFFFFFFFFFFFFFF
        x = (x ^ (x >> 7)) & 0xFFFFFFFFFFFFFFFF
        x = (x ^ (x << 17)) & 0xFFFFFFFFFFFFFFFF
        self.state = x
        
        return self.state / (2**64) # Normalize to [0, 1)

In [6]:
def MT19937(seed):
    """
    Mersenne Twister (MT19937) to generate a pseudo-random variable.  
    Returns a Generator object.
    """
    return np.random.Generator(bit_generator=np.random.MT19937(seed))

In [7]:
class RNG:
    """
    Random Number Generator (RNG)
    Use this class to generate a random number using some of the available generators:
    - xorshift64
    - mt19937
    - lcg
    The default algorithm is xorshift64.
    You can also pass a seed to the RNG. If no seed is provided, the current time in nanoseconds is used as the seed.
    The seed must be non-zero for xorshift64 and lcg algorithms.
    
    Usage:
    Without passing seed:
    r = RNG("xorshift64")
    r.random()
    
    Passing seed:
    r = RNG("xorshift64", seed=1234)
    r.random()
    """
    def __init__(self, algo='xorshift64', seed=None):
        if seed is None:
            seed = 1234
        
        self.algo = algo.lower()
        
        if self.algo == 'xorshift64':
            if seed == 0:
                raise ValueError("Seed must be non-zero for Xorshift64")
            self.generator = Xorshift64(seed)
        elif self.algo == 'mt19937':
            self.generator = MT19937(seed)
        elif self.algo == 'lcg':
            self.generator = LCG(seed)
        else:
            raise ValueError(f"Unknown algorithm: {self.algo}")
    
    def random(self):
        return self.generator.random()

#### Intensidad del proceso Poisson homogéneo
$$
    \lambda(t) = 30 + 30 \cdot \sin\left(\frac{2\pi t}{24}\right) \text{ (clientes/hora)}
$$

In [8]:
# Intensidad del Poisson proceso homogéneo
def lambda_t(t):
    """
    Intensity function for a homogeneous Poisson process.
    """
    return 30 + 30 * np.sin((2*np.pi*t)/24)


#### Tiempo de Atención

Corresponde a una distribución exponencial con tasa $ \mu = 40 $ (clientes / hora).  
Para calcular el tiempo de atención usamos la transformada inversa:
$$
    T = -\frac{1}{40} \ln{U}
$$

In [9]:
def service_time(gen: RNG):
    u = gen.random()
    return -np.log(u) / 40

#### Servidor

El servidor se encarga de procesar la lista de eventos t_arrivals.  
La función devuelve metricas sobre el uso y evolución del sistema a lo largo del tiempo:

- **Tiempo promedio en el sistema por cliente:**  
    $ \text{t\_end\_service}[i] - \text{t\_arrivals}[i] $
- **Tiempo de espera promedio en la cola:**  
    $ \text{t\_start\_service}[i] - \text{t\_arrivals}[i] $
- **Tiempo promedio de servicio:**  
    $ \text{t\_end\_service}[i] - \text{t\_start\_service}[i] $
- **Registro de variación de longitud de la cola a lo largo del tiempo:**  
    $ \text{events\_queue} $

In [10]:
from queuelib.queue import FifoMemoryQueue as Queue

def server(t_arrivals: list, gen: RNG):
    """
    Handles a list of users and returns metrics about its usage.  
    
    Parameters:
    - **t_arrivals**: List of events
    - **gen**: An object of type RNG
    """
    t_server_available = 0
    wait_queue = Queue()
    
    # metrics
    t_start_service = []
    t_end_service = []
    events_queue = []
    
    # process all events
    for t_curr, t_next in zip(t_arrivals, t_arrivals[1:]):
        t_arrival = t_curr
        
        if t_arrival >= t_server_available:
            # server is available
            t_start = t_arrival
            t_service = service_time(gen)
            t_end = t_start + t_service
            
            t_server_available = t_end
            
            # metrics
            t_start_service.append(t_start)
            t_end_service.append(t_end)
            events_queue.append((t_arrival, len(wait_queue)))
            events_queue.append((t_end, len(wait_queue)))

            # process events on queue
            while len(wait_queue) and t_server_available <= t_next:
                wait_queue.pop()
                t_start = t_server_available
                t_service = service_time(gen)
                t_end = t_start + t_service
            
                t_server_available = t_end
                
                # metrics
                t_start_service.append(t_start)
                t_end_service.append(t_end)
                events_queue.append((t_arrival, len(wait_queue)))
                events_queue.append((t_end, len(wait_queue)))
                
        else:
            # server is unavailable
            wait_queue.push(t_curr)
            
            #metrics
            events_queue.append((t_arrival, len(wait_queue)))

    return np.array(t_start_service), np.array(t_end_service), np.array(events_queue)

#### Generación de eventos

Generamos eventos con $ t \ \epsilon \ [0, \ T\_max] $.  
La variable `lambda_max` corresponde al máximo valor posible de la función `lambda_t` que es 60.
La generación de candidatos se basa en:  
1. Generamos el candidato `u1` y lo sumamos al tiempo transcurrido usando el método de la transformada inversa.
2. Si todavía no superamos la cota superior `T_max`, generamos `u2`.
3. Decidimos si `t` es un valor razonable para este momento usando la relación $ u2 \lt \frac{\lambda(t)}{60} $ que corresponde a la probabilidad de que se dé lo mencionado.
4. Aceptamos `t` y lo agregamos a la lista o seguimos con la siguiente iteración.

In [11]:
def events_generator(gen: RNG) -> list:
    t = 0
    arrivals = []
    T_max = 48
    lambda_max = 60
    
    while True:
        # Generate an interarrival time candidate
        u1 = gen.random()
        
        # Fix the log(0) issue by ensuring u1 is never exactly 1
        if u1 >= 1.0:
            u1 = 1.0 - 1e-15  # Very small epsilon to avoid log(0)
        
        t += -np.log(1 - u1) / lambda_max
        
        # Terminate if generated time is bigger than T_max bound
        if t > T_max:
            break
        
        # Decide if t is accepted based on lambda(t) / lambda_max relation
        u2 = gen.random()
        if u2 < lambda_t(t) / lambda_max:
            arrivals.append(t)
    
    return np.array(arrivals)


Simulamos durante 48 horas, con los 3 generadores de números aleatorios mencionados.
Simularemos 1_000 veces con distintas semillas para cada generador y luego tomaremos el promedio de las métricas obtenidas.

In [12]:
def run_simulation(algo='xorshift64', seed=None):
    gen = RNG(algo, seed)
    arrivals = events_generator(gen)
    
    t_start_service, t_end_service, events_queue = server(arrivals, gen)
    
    return np.array(t_start_service), np.array(t_end_service), np.array(events_queue)

In [13]:
def calculate_mean(arrays_list):
    if not arrays_list:
        return np.array([])
    # Find the maximum length among the arrays
    max_len = max(len(arr) for arr in arrays_list)
    
    # Create an array NaNs
    arrays = []
    for arr in arrays_list:
        # Create an array full of NaN of the maximum size
        nan_array = np.full(max_len, np.nan)
        # Copy the original values
        nan_array[:len(arr)] = arr
        arrays.append(nan_array)
    
    # Convert to matrix and calculate mean ignoring NaN
    matrix = np.array(arrays)
    return np.nanmean(matrix, axis=0)

In [14]:
np.random.seed(42) 
seeds = np.random.randint(1, 10_000, size=1_000)  # Generate 1_000 random seeds
# Initialize an empty array for simulations
start_service_simulations = np.array([])  
end_service_simulations = np.array([])  
events_queue_simulations = np.array([])  
generators = ["Xorshift64", "MT19937", "LCG"]
for i, algo in enumerate(generators):
    all_t_start_service = np.array([])
    all_t_end_service = np.array([])
    all_events_queue = np.array([])
    for seed in tqdm(seeds):
        seed = int(seed)
        t_start_service, t_end_service, events_queue = run_simulation(algo, seed)
        all_t_start_service = np.append(all_t_start_service, t_start_service)
        all_t_end_service = np.append(all_t_end_service, t_end_service)
        all_events_queue = np.append(all_events_queue, events_queue.flatten())
    
    # calculate the mean for each algorithm
    mean_t_start_service = calculate_mean([all_t_start_service])
    mean_t_end_service = calculate_mean([all_t_end_service])
    mean_events_queue = calculate_mean([all_events_queue])
    # Append the results to the simulations array
    start_service_simulations = np.append(start_service_simulations, mean_t_start_service)
    end_service_simulations = np.append(end_service_simulations, mean_t_end_service)
    events_queue_simulations = np.append(events_queue_simulations, mean_events_queue)


  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [02:52<00:00,  5.81it/s]


In [19]:
# Plotting to visualize the results and compare the algorithms